In [12]:
from firedrake import *
from pyadjoint import *
import matplotlib.pyplot as plt
from cyipopt import *
import numpy as np

In [13]:
E, nu = 1e5, 0.3
L, H = 3.0, 1.0
F = 2000
p, eps = Constant(3.0), Constant(1.0e-3)
rho_0, Vol = Constant(0.5), Constant(0.5*L*H)

nx, ny = 300, 100
mesh = RectangleMesh(nx, ny, L, H)

V, W = VectorFunctionSpace(mesh, "CG", 2), FunctionSpace(mesh, "CG", 1)

bc = [DirichletBC(V, Constant((0, 0)), 1)]

# Aplicación de carga
t = Constant((0.0, -F))

def forward(rho):
    u, v =TrialFunction(V), TestFunction(V)
    E_rho = eps +(E-eps) * (rho**p)
    lmbda = nu * E_rho /((1+nu) * (1-2*nu))
    mu = E_rho/(2*(1+nu))
    a = 2 * mu * inner(sym(grad(u)), sym(grad(v))) * dx + lmbda * div(u) * div(v) * dx
    L = inner(t, v)*ds(2)
    u = Function(V, name="Displacement")
    solve(a == L, u, bc, annotate=True)
    return u

In [17]:
rho = interpolate(Constant(float(Vol)), W)
u = forward(rho)
J = assemble(inner(t, u) * ds(2))
m = Control (rho)
Jhat = ReducedFunctional(AdjFloat(J), m)
constraint = {"type": "ineq", "fun": lambda m: rho - assemble(m * dx)}

lb, ub = 0.0, 1.0

problem = MinimizationProblem(Jhat, bounds=(lb, ub))
parameters = {"acceptable_tol": 1.0e-5, "maximum_iterations": 100}
solver = IPOPTSolver(problem, parameters)

shape_opt = solver.solve()

/home/brandon/myenv/lib/python3.12/site-packages/firedrake/interpolation.py:391: FutureWarning: The use of `interpolate` to perform the numerical interpolation is deprecated.
This feature will be removed very shortly.

Instead, import `interpolate` from the `firedrake.__future__` module to update
the interpolation's behaviour to return the symbolic `ufl.Interpolate` object associated
with this interpolation.

You can then assemble the resulting object to get the interpolated quantity
of interest. For example,

```
from firedrake.__future__ import interpolate
...

assemble(interpolate(expr, V))
```

Alternatively, you can also perform other symbolic operations on the interpolation operator, such as taking
the derivative, and then assemble the resulting form.

  warnings.warn("""The use of `interpolate` to perform the numerical interpolation is deprecated.



List of options:

                                    Name   Value                # times used
                          acceptable_tol = 1e-05                     1
                   hessian_approximation = limited-memory            7
                                max_iter = 100                       1
                             print_level = 6                         2

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.17, running with linear solver ma27.

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian